---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [76]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [77]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [78]:
import pandas as pd

def get_list_of_university_towns():
    file = 'university_towns.txt'
    
    data=pd.read_csv(file, names=['data'],sep='\n')
    data.data = data.data.replace(["\n|\r"],[""],regex=True)
    #data.data = data.data.replace(["\(.*\)"],[""],regex=True)
    data.data = data.data.replace(["\[.*\]"],[""],regex=True)
    #data.data = data.data.replace(["\s\(.*"],[""],regex=True)
    data.data = data.data.str.split('(').apply(lambda x:  x[0])
    data.data = data.data.str.strip()
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
     
    # for Python 3 wrap states.items in the list
    states = pd.DataFrame(list(states.items()), columns=['State_abbr', 'State_full'])
    DataFrame = pd.merge(data,states,how="left",left_on="data",right_on="State_full")
     
     # cool method
    DataFrame = DataFrame.fillna(method='ffill')
    DataFrame = DataFrame[DataFrame['data'] != DataFrame['State_full']]
    vars=['data','State_full']
    DataFrame=DataFrame[vars]
    vars=['RegionName','State']
    DataFrame.columns = vars
    DataFrame = DataFrame[['State','RegionName']]
    
    # these have to be manually fixed since some towns have state names. Only three cases are identified
    to_add = pd.DataFrame([['California','California'],['Delaware', 'Delaware'],['Indiana', 'Indiana']], columns=['State','RegionName'], index = [413,371,426])  
    
    DataFrame=DataFrame.append(to_add)   
    DataFrame.index=range(DataFrame.shape[0])    
    return DataFrame


In [79]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
 
    file = 'gdplev.xls'

    names = ['quarter', 'gdp_current', 'gp_constant']    

    data=pd.read_excel(file, skiprows = 220,parse_cols='E:G',header=None, names=names)
    data['diff'] = data['gp_constant'].pct_change()
    data['label']= data['diff'].map(lambda x: 1 if x>0  else 0 ) 
    #data['year'] = data.quarter.str[:4]

    temp=data.label.astype(str).str.cat()
    temp = temp.index('000011')
    temp =  pd.Series(range(temp, temp + 4))
    temp = data.iloc[temp]

    temp = temp.iloc[0:1,0:1].to_string(index=False,header=False)

    return temp

get_recession_start()

'2008q3'

In [80]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    file = 'gdplev.xls'

    names = ['quarter', 'gdp_current', 'gp_constant']    

    data=pd.read_excel(file, skiprows = 220,parse_cols='E:G',header=None, names=names)
    data['diff'] = data['gp_constant'].pct_change()
    data['label']= data['diff'].map(lambda x: 1 if x>0  else 0 ) 
    #data['year'] = data.quarter.str[:4]

    temp=data.label.astype(str).str.cat()
    temp = temp.index('000011')
    temp =  pd.Series(range(temp, temp + 6))
    temp = data.iloc[temp]

    temp = temp.iloc[5:6,0:1].to_string(index=False,header=False)

    return temp

get_recession_end()

'2009q4'

In [81]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    file = 'gdplev.xls'

    names = ['quarter', 'gdp_current', 'gp_constant']    

    data=pd.read_excel(file, skiprows = 220,parse_cols='E:G',header=None, names=names)
    data['diff'] = data['gp_constant'].pct_change()
    data['label']= data['diff'].map(lambda x: 1 if x>0  else 0 ) 
    #data['year'] = data.quarter.str[:4]

    temp=data.label.astype(str).str.cat()
    temp = temp.index('000011')
    temp =  pd.Series(range(temp, temp + 6))
    temp = data.iloc[temp]

    bottom = temp.gp_constant.idxmin()
    temp = temp.loc[37,['quarter']].to_string(index=False,header=False)

    return temp

get_recession_bottom() 

'2009q2'

In [96]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''  

    file = 'City_Zhvi_AllHomes.csv'
    
    data = pd.read_csv(file,index_col=['RegionID', 'RegionName','State'])
    data = data.loc[:, '2000-01':'2016-08']
    cols = data.columns.values.tolist()

    def convert_to_quarter(d):
        ts = pd.Timestamp(pd.to_datetime(d,format='%Y-%m'))
        return "{}Q{}".format(ts.year, ts.quarter)

    # Python 3 map wrap in list
    cols_new = list(map(convert_to_quarter,cols))
    cols_new =  [x.lower() for x in cols_new]

    # to replace columns, create a new dict with old and new names
    col_new_dict = {i:j for i,j in zip(cols,cols_new)}
    # and then just use rename method
    data.rename(columns=col_new_dict, inplace=True)

    data=data.reset_index()
    data = pd.melt(data,id_vars=['RegionID', 'RegionName','State'])
    #grouped = data.stack()
    #grouped = data.groupby(['State','RegionName','variable'])['value'].reset_index()

    #temp = pd.melt(data,id_vars=['RegionName','State'])
    temp = pd.pivot_table(data, values='value', index=['RegionID','RegionName', 'State'], columns=['variable'], aggfunc=np.mean).reset_index()

    temp['myState'] = temp['State']
    temp['myState'] = temp['State'].replace(states, inplace=True)
    temp.columns.name = None
    temp.set_index(['State','RegionName'],inplace=True)
    temp = temp.drop('RegionID', 1)
    temp = temp.drop('myState', 1)
    
    return temp

In [106]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    #university_towns = get_list_of_university_towns()
    
    #housing_df = convert_housing_data_to_quarters()
    
    prices_begin = housing_df['2008q2']
    prices_end = housing_df['2009q4']
    
    housing_df['ratio'] = prices_begin/prices_end #The higher this ratio, the worse
    
    university_towns = university_towns.set_index(['State', 'RegionName'])
    
    ratio_college = housing_df.loc[list(university_towns.index)]['ratio'].dropna()
    
    ratio_not_college_indices = set(housing_df.index) - set(ratio_college.index)
    
    ratio_not_college = housing_df.loc[list(ratio_not_college_indices),:]['ratio'].dropna()
        
    statistic, p_value = tuple(ttest_ind(ratio_college, ratio_not_college))
    
    outcome = statistic < 0
    
    different = p_value < 0.01
    
    better = ["non-university town", "university town"]
    
    return (different, p_value, better[outcome])

run_ttest()

(True, 0.007863565232848908, 'university town')